# 🫔 Notion MCP Agent

# 1. ⚙️ Setup

In [1]:
from utilities.setup import *

In [2]:
GOOGLE_API_KEY = get_google_api_key()
retry_config = get_retry_config()
NOTION_API_KEY = get_notion_credentials()

✅ Gemini API key setup complete.
✅ Notion API key setup complete.


# 2. 🧑‍💻 Define Kaggle MCP Agent

In [3]:
mcp_notion_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",
            args=[
                "-y",
                "@notionhq/notion-mcp-server"
            ],
            env={
                "NOTION_TOKEN": NOTION_API_KEY,
            }
        ),
        timeout=60
    )
)

print("✅ MCP Notion Toolset defined.")

✅ MCP Notion Toolset defined.


In [4]:
notion_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="notion_agent",
    instruction=(
        "You are a Notion assistant. "
        "CRITICAL INSTRUCTIONS FOR READING CONTENT: "
        "1. To read a page, use the tool 'API-get-block-children'. "
        "2. The tool input argument is 'block_id' (pass the page_id here). "
        "3. **DATA EXTRACTION RULE**: The output is a JSON list of blocks. "
        "   Text is hidden deep inside. You must iterate through 'results', "
        "   check fields like 'paragraph', 'heading_1', 'bulleted_list_item', etc., "
        "   and extract the 'plain_text' from within the 'rich_text' array. "
        "   Do not say the page is empty just because you see JSON wrappers. Dig for the 'plain_text'."
        "4. If 'API-post-search' returns nothing, ask the user to 'Connect' the page to the integration."
    ),
    tools=[mcp_notion_server],
)

In [5]:
runner = InMemoryRunner(agent=notion_agent)

In [8]:
# 1. Create a runner specifically for the Notion Agent
# (Assuming you are using the same Runner class as before, likely 'McpRunner' or similar from your imports)
# notion_runner = McpRunner(notion_agent)

# 2. Now run the debug command on the *notion_runner*
# Notice we do NOT pass 'agent=' here anymore.
response = await runner.run_debug(
    "Read the content of page ID '2b5c2851a6528022b0fac4f022403865' using API-get-block-children. Extract and summarize the text.",
    verbose=True
)


 ### Continue session: debug_session_id

User > Read the content of page ID '2b5c2851a6528022b0fac4f022403865' using API-get-block-children. Extract and summarize the text.
notion_agent > The content of the page includes several to-do items and a toggle block with tips.

Here's a summary:
- Create an account with Notion.
- Instructions on how to type '/' to access various block options like headers, tables, and to-do lists.
- Tips on using the sidebar to find, organize, and add new pages.
- A reminder to check the "Todo List" in the sidebar for more tips and tricks.
- A toggle block with the message: "This is a toggle block. Click the little triangle to see a few more useful tips!"


In [9]:
response

[Event(model_version='gemini-2.5-flash-lite', content=Content(
   parts=[
     Part(
       text="""The content of the page includes several to-do items and a toggle block with tips.
 
 Here's a summary:
 - Create an account with Notion.
 - Instructions on how to type '/' to access various block options like headers, tables, and to-do lists.
 - Tips on using the sidebar to find, organize, and add new pages.
 - A reminder to check the "Todo List" in the sidebar for more tips and tricks.
 - A toggle block with the message: "This is a toggle block. Click the little triangle to see a few more useful tips!""""
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   cache_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=7827


🎉 Completed the **🕵🏻‍♂️ Notion MCP Agent**.